# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
offer = pd.read_json('data/portfolio.json', orient='records', lines=True)
people = pd.read_json('data/profile.json', orient='records', lines=True)
record = pd.read_json('data/transcript.json', orient='records', lines=True)

#  1. Data Preprocessing and Feature Engineering

## 1. data preview

In [2]:
offer.head(2)

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10


In [3]:
people.head(2)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0


In [4]:
record.head(2)

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}


## 2. data propoessing and wrangling

offer table: release channels into categorical columns

In [5]:
offer.channels.max()

['web', 'email', 'mobile', 'social']

In [6]:
offer['web'] = offer['channels'].apply(lambda x: 1 if 'web' in x else 0)
offer['email'] = offer['channels'].apply(lambda x: 1 if 'email' in x else 0)
offer['mobile'] = offer['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
offer['social'] = offer['channels'].apply(lambda x: 1 if 'social' in x else 0)

In [7]:
offer.drop(['channels'], axis=1, inplace=True)
offer.rename(columns={'id':'offer_id'}, inplace=True)

In [8]:
offer.head(2)

,difficulty,duration,offer_id,offer_type,reward,web,email,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,0,1,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1


In [9]:
offer.dtypes

difficulty     int64
duration       int64
offer_id      object
offer_type    object
reward         int64
web            int64
email          int64
mobile         int64
social         int64
dtype: object

people table: get customer tenure (in months)

In [10]:
people['start_date'] = people['became_member_on'].astype(str).apply(lambda x : pd.to_datetime(x, infer_datetime_format=True))
people['cust_tenure'] = people['start_date'].apply(lambda x: round((pd.to_datetime('2023-02-21') - x)/np.timedelta64(1, 'M')),1)

In [11]:
people.drop(['became_member_on'], axis=1, inplace=True)
people.rename(columns={'id':'person_id'}, inplace=True)

In [12]:
people.head(2)

,age,gender,person_id,income,start_date,cust_tenure
0,118,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,72
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,67


In [13]:
#missing rate of 'income'
people[people.income.isna()==True].shape[0]/people.income.shape[0]

0.12794117647058822

In [14]:
people.dtypes

age                     int64
gender                 object
person_id              object
income                float64
start_date     datetime64[ns]
cust_tenure             int64
dtype: object

record table: release offer_id into columns

In [15]:
record['offer_id'] = record['value'].apply(lambda x: x['offer id'] if 'offer id' in x.keys()  else np.nan)
record['trxn_amt'] = record['value'].apply(lambda x: x['amount'] if 'amount' in x.keys()  else np.nan)

In [16]:
record.drop(['value'], axis=1, inplace=True)
record.rename(columns={'person':'person_id'}, inplace=True)

## 3. generate main table:
    - record_m table: record table + 'if_viewed', 'if_completed'
        - if_viewed: 1 if this client viewed offer, 0 if not
        - if_completed: 1 if this offer_id come with a 'offer_completed' event, 0 elsewise
        - if_valid: if this completed offer is drived by offer: = if_viewed & if_completed, and t_view < t_trxn
    - main: big table, record_m table joined with offer and person infomation
       
       

#### 1. Link offer_id to 'offer_complete' record

In [17]:
record_p = record.groupby('person_id').apply(lambda x: x.sort_values('time', ascending=True))

In [18]:
record_p.columns

Index(['event', 'person_id', 'time', 'offer_id', 'trxn_amt'], dtype='object')

In [19]:
record_p.index.names

FrozenList(['person_id', None])

In [20]:
record_p.index = record_p.index.droplevel(level=1)

In [21]:
#fill offer_id
df_fill = record_p[record_p['event'].isin(['offer received','offer completed'])]
df_rest = record_p[~record_p['event'].isin(['offer received','offer completed'])]

df_filled = df_fill.ffill()
record_f = pd.concat([df_filled, df_rest], axis=0)

In [22]:
record_f = record_f.sort_values(['person_id','time'], ascending=True)
record_f = record_f.reset_index(drop=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'person_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


#### 2. create record_m table

In [23]:
record_f.head(2)

,event,person_id,time,offer_id,trxn_amt
0,offer received,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,NaN
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,5a8bc65990b245e5a138643cd4eb9837,NaN


In [24]:
base = record_f[record_f['event']=='offer received'][['person_id','time','offer_id']].set_index(['person_id','offer_id'])

view = record_f[record_f['event']=='offer viewed'][['person_id','time','offer_id']].set_index(['person_id','offer_id'])

completed = record_f[record_f['event']=='offer completed'][['person_id','time','offer_id']].set_index(['person_id','offer_id'])

In [25]:
record_m_raw = base.join(view, how='left', lsuffix='_rcv',rsuffix='_view').join(completed, how='left')

In [26]:
record_m_raw.rename(columns={'time':'time_cplt'},inplace=True)

In [27]:
record_m_raw = record_m_raw.reset_index()

In [28]:
#add col: if view --> 1 viewed, 0 elsewise
record_m_raw['if_view'] = record_m_raw['time_view'].notna().apply(np.int)

In [29]:
#add col: if complete --> 1 completed offer, 0 elsewise
record_m_raw['if_complete'] = record_m_raw['time_cplt'].notna().apply(np.int)

In [30]:
#add col: if_valid --> 1: the completed offer is valid, meaing the person viewed the offer before complete it, 0 elsewise
record_m_raw['if_valid'] =  record_m_raw.apply(lambda x: 1 if x['if_complete']==1 and x['if_view']==1 and x['time_view']<=x['time_cplt'] else 0, axis=1)

In [31]:
#get record_m table
record_m = record_m_raw.drop_duplicates()

#### 3. attach offer detail, person detail

In [32]:
main = record_m.merge(people, on='person_id', how='left').merge(offer, on='offer_id', how='left')

In [33]:
main.head(10)

,person_id,offer_id,time_rcv,time_view,time_cplt,if_view,if_complete,if_valid,age,gender,...,start_date,cust_tenure,difficulty,duration,offer_type,reward,web,email,mobile,social
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576,NaN,576.0,0,1,0,33,M,...,2017-04-21,70,10,7,discount,2,1,1,1,0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,372.0,NaN,1,0,0,33,M,...,2017-04-21,70,0,4,informational,0,1,1,1,0
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,192.0,NaN,1,0,0,33,M,...,2017-04-21,70,0,3,informational,0,0,1,1,1
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,1,1,0,33,M,...,2017-04-21,70,5,5,bogo,5,1,1,1,1
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,1,1,0,33,M,...,2017-04-21,70,10,10,discount,2,1,1,1,1
5,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,168,216.0,NaN,1,0,0,118,None,...,2018-04-25,58,5,5,bogo,5,1,1,1,1
6,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,168,630.0,NaN,1,0,0,118,None,...,2018-04-25,58,5,5,bogo,5,1,1,1,1
7,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,576,216.0,NaN,1,0,0,118,None,...,2018-04-25,58,5,5,bogo,5,1,1,1,1
8,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,576,630.0,NaN,1,0,0,118,None,...,2018-04-25,58,5,5,bogo,5,1,1,1,1
9,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,NaN,1,0,0,40,O,...,2018-01-09,61,20,10,discount,5,1,1,0,0


# 2. EDA
have some insights of the data

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt

In [60]:
sns.set(style = 'whitegrid')

In [ ]:
'if_complete':('complete_rate','mean'),'if_valid':('valid_rate','mean')

#### 1. see the response rate by gender & offer_type

In [48]:
resp_1 = main.groupby(['gender', 'offer_type']).agg({'if_view':['mean','count'],'if_complete':['mean','count'],'if_valid':['mean','count']})

In [53]:
resp_1.columns = ['_'.join(col) for col in resp_1.columns.values]

In [56]:
resp_1 = resp_1.reset_index()

In [58]:
resp_1

,gender,offer_type,if_view_mean,if_view_count,if_complete_mean,if_complete_count,if_valid_mean,if_valid_count
0,F,bogo,0.913801,18318,0.773665,18318,0.513375,18318
1,F,discount,0.827095,17420,0.769059,17420,0.493858,17420
2,F,informational,0.795681,6715,0.111095,6715,0.062695,6715
3,M,bogo,0.902886,23385,0.617490,23385,0.413171,23385
4,M,discount,0.803779,23025,0.648817,23025,0.438350,23025
5,M,informational,0.789886,9314,0.102856,9314,0.064204,9314
6,O,bogo,0.937500,544,0.748162,544,0.527574,544
7,O,discount,0.891667,600,0.745000,600,0.523333,600
8,O,informational,0.882353,255,0.192157,255,0.101961,255


In [62]:
! pip install --upgrade seaborn

    100% |████████████████████████████████| 296kB 12.8MB/s ta 0:00:01
    100% |████████████████████████████████| 13.4MB 1.3MB/s eta 0:00:01  4% |█▍                              | 563kB 23.6MB/s eta 0:00:01    20% |██████▌                         | 2.7MB 20.5MB/s eta 0:00:01    45% |██████████████▊                 | 6.1MB 25.9MB/s eta 0:00:01    71% |███████████████████████         | 9.6MB 23.9MB/s eta 0:00:01    87% |████████████████████████████    | 11.7MB 17.4MB/s eta 0:00:01
    100% |████████████████████████████████| 11.5MB 3.6MB/s eta 0:00:01  3% |█                               | 358kB 23.9MB/s eta 0:00:01    31% |██████████                      | 3.6MB 24.0MB/s eta 0:00:01    40% |█████████████                   | 4.7MB 22.8MB/s eta 0:00:01    49% |████████████████                | 5.8MB 19.7MB/s eta 0:00:01    69% |██████████████████████▏         | 8.0MB 22.2MB/s eta 0:00:01
    39% |████████████▌                   | 19.3MB 22.5MB/s eta 0:00:02  3% |█▎                         

In [61]:
g = sns.catplot(
    data = resp_1, kind = 'bar',
    x='offer_type', y = 'if_view_mean', hue='gender',
    errorbar = 'sd', palette = 'dark', height = 6
)
g.despine(left = True)
g.set_axis_labels("","offer view rate")
g.legend.set_title("Offer view rate by gender and offer_type")

AttributeError: module 'seaborn' has no attribute 'catplot'

# 3. Problem statement

Now we have a main table with cleaned, labeled data, we want to know what demographic group is most worthy sending offer to.
This task will break down into few steps:
    - 1. A classification model to predict whether a client will respond to an offer
    - 2. On top of the classification model, we add another layer to grade down those clients who will make transaction no matter what.
   
They will be further broken down in below.

## 1. classification model
#### 1. get the right features
#### 2. use DNN as classifier

## 2. Exclude clients who buy no matter what
#### 1. See if a cluster can clearly cut them out
#### 2. The final offer_send list should remove clients appear in this cluster.